<a href="https://colab.research.google.com/github/RahafSobh/RahafSobh/blob/main/Hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#אפשרות 1

import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

URLS = [
    "https://edition.cnn.com/2025/12/22/media/60-minutes-cecot-bari-weiss-canada-global-tv?iid=cnn_buildContentRecirc_end_recirc&recs_exp=most-popular-article-end&tenant_id=popular.en",
    "https://www.nhm.ac.uk/visit/exhibitions/wildlife-photographer-of-the-year.html",
    "https://is-web.hevra.haifa.ac.il/images/2025_SEM._aa.pdf",
]

def manual_parse_url(url: str):

    scheme = None
    host_port = None
    host = None
    port = None
    path = "/"
    query_str = ""
    file_name = None


    if "://" in url:
        scheme, rest = url.split("://", 1)
    else:
        rest = url

    if "?" in rest:
        rest, query_str = rest.split("?", 1)

    if "/" in rest:
        host_port, path = rest.split("/", 1)
        path = "/" + path
    else:
        host_port = rest
        path = "/"

    if ":" in host_port:
        host, port_str = host_port.split(":", 1)
        try:
            port = int(port_str)
        except ValueError:
            port = None
    else:
        host = host_port

    if path and path != "/":
        last = path.split("/")[-1]
        if last != "":
            file_name = last

    subdomain = None
    domain = None
    tld = None
    if host:
        parts = host.split(".")
        if len(parts) >= 2:
            tld = parts[-1]
            domain = parts[-2]
            if len(parts) > 2:
                subdomain = ".".join(parts[:-2])
        else:
            domain = host


    query_pairs = {}
    if query_str.strip() != "":
        pairs = query_str.split("&")
        for p in pairs:
            if "=" in p:
                k, v = p.split("=", 1)
            else:
                k, v = p, ""

            if k in query_pairs:
                if isinstance(query_pairs[k], list):
                    query_pairs[k].append(v)
                else:
                    query_pairs[k] = [query_pairs[k], v]
            else:
                query_pairs[k] = v

    return {
        "url": url,
        "scheme": scheme,
        "tld": tld,
        "domain": domain,
        "subdomain": subdomain,
        "host": host,
        "port": port,
        "path": path,
        "file_name": file_name,
        "query_pairs": query_pairs
    }

def fetch_text(url: str, timeout=10):
    try:
        req = Request(url, headers={"User-Agent": "Mozilla/5.0 (student-bot/1.0)"})
        with urlopen(req, timeout=timeout) as resp:
            return resp.read().decode("utf-8", errors="replace")
    except (HTTPError, URLError):
        return None
    except Exception:
        return None

def parse_robots_manual(text: str):
    if not text:
        return [], [], None

    disallow_paths = []
    user_agents = []
    crawl_delay = None
    current_ua = None

    for line in text.splitlines():
        line = line.strip()
        if line == "" or line.startswith("#"):
            continue
        if "#" in line:
            line = line.split("#", 1)[0].strip()

        if ":" not in line:
            continue

        key, value = line.split(":", 1)
        key = key.strip().lower()
        value = value.strip()

        if key == "user-agent":
            current_ua = value
            user_agents.append(value)

        elif key == "disallow":
            if value != "":
                disallow_paths.append(value)

        elif key == "crawl-delay":

            try:
                crawl_delay = float(value)
            except ValueError:
                pass

    def uniq(lst):
        out = []
        for x in lst:
            if x not in out:
                out.append(x)
        return out

    return uniq(disallow_paths), uniq(user_agents), crawl_delay

def robots_for_url(info):

    if not info["scheme"] or not info["host"]:
        return {
            "robots_url": None,
            "robots_exists": False,
            "robots_disallow_full_urls": [],
            "robots_user_agents": [],
            "robots_crawl_delay": None
        }

    base = f'{info["scheme"]}://{info["host"]}'
    if info["port"] is not None:
        base += f':{info["port"]}'

    robots_url = base + "/robots.txt"
    text = fetch_text(robots_url)

    if text is None:
        return {
            "robots_url": robots_url,
            "robots_exists": False,
            "robots_disallow_full_urls": [],
            "robots_user_agents": [],
            "robots_crawl_delay": None
        }

    disallow_paths, user_agents, crawl_delay = parse_robots_manual(text)

    disallow_full = []
    for p in disallow_paths:
        if not p.startswith("/"):
            p = "/" + p
        disallow_full.append(base + p)

    return {
        "robots_url": robots_url,
        "robots_exists": True,
        "robots_disallow_full_urls": disallow_full,
        "robots_user_agents": user_agents,
        "robots_crawl_delay": crawl_delay
    }

rows = []
for u in URLS:
    info = manual_parse_url(u)
    robots = robots_for_url(info)
    rows.append({**info, **robots})

df = pd.DataFrame(rows)
display(df)


for _, r in df.iterrows():
    print("\n" + "="*70)
    print("URL:", r["url"])
    print("TLD:", r["tld"], "| domain:", r["domain"], "| subdomain:", r["subdomain"])
    print("path:", r["path"], "| file:", r["file_name"], "| port:", r["port"])
    print("query:", r["query_pairs"])
    print("robots:", r["robots_url"], "| exists?", r["robots_exists"])
    if r["robots_exists"]:
        print("disallow (full):")
        for x in r["robots_disallow_full_urls"]:
            print(" -", x)
        print("user-agents:", r["robots_user_agents"])
        print("crawl-delay:", r["robots_crawl_delay"])


,url,scheme,tld,domain,subdomain,host,port,path,file_name,query_pairs,robots_url,robots_exists,robots_disallow_full_urls,robots_user_agents,robots_crawl_delay
0,https://edition.cnn.com/2025/12/22/media/60-mi...,https,com,cnn,edition,edition.cnn.com,None,/2025/12/22/media/60-minutes-cecot-bari-weiss-...,60-minutes-cecot-bari-weiss-canada-global-tv,"{'iid': 'cnn_buildContentRecirc_end_recirc', '...",https://edition.cnn.com/robots.txt,True,"[https://edition.cnn.com/, https://edition.cnn...","[AI2Bot, Ai2Bot-Dolma, Amazonbot, amzn-searchb...",NaN
1,https://www.nhm.ac.uk/visit/exhibitions/wildli...,https,uk,ac,www.nhm,www.nhm.ac.uk,None,/visit/exhibitions/wildlife-photographer-of-th...,wildlife-photographer-of-the-year.html,{},https://www.nhm.ac.uk/robots.txt,True,"[https://www.nhm.ac.uk/uksf-bin/, https://www....",[*],8.0
2,https://is-web.hevra.haifa.ac.il/images/2025_S...,https,il,ac,is-web.hevra.haifa,is-web.hevra.haifa.ac.il,None,/images/2025_SEM._aa.pdf,2025_SEM._aa.pdf,{},https://is-web.hevra.haifa.ac.il/robots.txt,True,[https://is-web.hevra.haifa.ac.il/administrato...,[*],NaN



URL: https://edition.cnn.com/2025/12/22/media/60-minutes-cecot-bari-weiss-canada-global-tv?iid=cnn_buildContentRecirc_end_recirc&recs_exp=most-popular-article-end&tenant_id=popular.en
TLD: com | domain: cnn | subdomain: edition
path: /2025/12/22/media/60-minutes-cecot-bari-weiss-canada-global-tv | file: 60-minutes-cecot-bari-weiss-canada-global-tv | port: None
query: {'iid': 'cnn_buildContentRecirc_end_recirc', 'recs_exp': 'most-popular-article-end', 'tenant_id': 'popular.en'}
robots: https://edition.cnn.com/robots.txt | exists? True
disallow (full):
 - https://edition.cnn.com/
 - https://edition.cnn.com/*.jsx$
 - https://edition.cnn.com/*.jsx$
 - https://edition.cnn.com/*.jsx/
 - https://edition.cnn.com/*.jsx?
 - https://edition.cnn.com/ads/
 - https://edition.cnn.com/aol/
 - https://edition.cnn.com/api/
 - https://edition.cnn.com/beta/
 - https://edition.cnn.com/browsers/
 - https://edition.cnn.com/cl/
 - https://edition.cnn.com/cnews/
 - https://edition.cnn.com/cnn_adspaces
 - http